In [1]:
from dataFormating import loadData,quantitate, historySplit, flagedAdresses
from keras.models import Sequential
from keras.layers import Dense
import keras
from keras.utils.vis_utils import plot_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

2023-12-29 13:52:07.940562: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-29 13:52:08.224285: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-29 13:52:08.258083: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-29 13:52:08.258102: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

DATA FORMATING

In [ ]:
paymentData = loadData("paymentHistory.csv")

paymentTrain, isFraudTrain, paymentTest, isFraudTest = historySplit(paymentData,0.2)
flaged = flagedAdresses(paymentTrain,isFraudTrain)


In [ ]:
paymentTrain = quantitate(paymentTrain,flaged)
paymentTest = quantitate(paymentTest,flaged)

             amount  oldbalanceOrg  newbalanceOrig  oldbalanceDest  \
1272524     3100.54       41219.24         38118.7            0.00   
1272525    48346.55       38118.70             0.0            0.00   
1272526    43508.13           0.00             0.0            0.00   
1272527     5282.07           0.00             0.0            0.00   
1272528     9748.18           0.00             0.0            0.00   
...             ...            ...             ...             ...   
6362615   339682.13      339682.13             0.0            0.00   
6362616  6311409.28     6311409.28             0.0            0.00   
6362617  6311409.28     6311409.28             0.0        68488.84   
6362618   850002.52      850002.52             0.0            0.00   
6362619   850002.52      850002.52             0.0      6510099.11   

         newbalanceDest  transactionTypeCode  originPastFraud  \
1272524            0.00                    1                0   
1272525            0.00      

In [ ]:
paymentTrain

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,transactionTypeCode,originPastFraud,destinationPastFraud
1272524,3100.54,41219.24,38118.7,0.00,0.00,1,0,0
1272525,48346.55,38118.70,0.0,0.00,0.00,1,0,0
1272526,43508.13,0.00,0.0,0.00,0.00,1,0,0
1272527,5282.07,0.00,0.0,0.00,0.00,1,0,0
1272528,9748.18,0.00,0.0,0.00,0.00,1,0,0
...,...,...,...,...,...,...,...,...
6362615,339682.13,339682.13,0.0,0.00,339682.13,2,1,1
6362616,6311409.28,6311409.28,0.0,0.00,0.00,3,1,1
6362617,6311409.28,6311409.28,0.0,68488.84,6379898.11,2,1,1
6362618,850002.52,850002.52,0.0,0.00,0.00,3,1,1


In [ ]:
isFraudTest

0          0
1          0
2          1
3          1
4          0
          ..
1272519    0
1272520    0
1272521    0
1272522    0
1272523    0
Name: isFraud, Length: 1272524, dtype: int64

Model 1 : neural network using keras

In [ ]:
model = Sequential()

model.add(Dense(16, input_dim=8, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.summary()

optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 16)                144       
                                                                 
 dense_7 (Dense)             (None, 32)                544       
                                                                 
 dense_8 (Dense)             (None, 2)                 66        
                                                                 
Total params: 754
Trainable params: 754
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(paymentTrain,isFraudTrain,epochs=2,batch_size=100)

Epoch 1/2
50901/50901 [==============================] - 70s 1ms/step - loss: 101.3779 - accuracy: 0.9886
Epoch 2/2
50901/50901 [==============================] - 76s 1ms/step - loss: 0.3277 - accuracy: 0.9986


In [ ]:
model.save("thisModel.keras")

In [ ]:
score = model.evaluate(paymentTest,isFraudTest)

print('Model accuracy:', score[1])

39767/39767 [==============================] - 60s 2ms/step - loss: 0.3573 - accuracy: 0.9988
Model accuracy: 0.9987897872924805


In [ ]:
Y_predictions = model.predict(paymentTest)

NameError: name 'model' is not defined

In [ ]:
print(Y_predictions)

[[0.0000000e+00 0.0000000e+00]
 [2.1465492e-05 8.0390137e-08]
 [2.1465492e-05 8.0390137e-08]
 ...
 [2.1465492e-05 8.0390137e-08]
 [2.1465492e-05 8.0390137e-08]
 [2.1465492e-05 8.0390137e-08]]


In [ ]:
actualPreds = []
for pred in Y_predictions:
    if pred[0] > pred[1]:
        actualPreds.append(0)
    elif pred[0] < pred[1]:
        actualPreds.append(1)
    else:
        actualPreds.append(0)


In [ ]:
good=0
bad=0

for i in range(len(isFraudTest)):
    if isFraudTest[i] == actualPreds[i]:
        good+=1
    else:
        print(isFraudTest[i],actualPreds[i])
        bad+=1

1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0


In [ ]:
print(good)

1270984


In [ ]:
print(bad)

1540


In [ ]:
(1270984*100) / 1272524

99.87898067148438

Model 2 : Decision tree

In [ ]:
print(isFraudTrain)

1272524    0
1272525    0
1272526    0
1272527    0
1272528    0
          ..
6362615    1
6362616    1
6362617    1
6362618    1
6362619    1
Name: isFraud, Length: 5090096, dtype: int64


In [ ]:
tree = RandomForestClassifier(n_estimators=100,random_state=42)
tree.fit(paymentTrain,isFraudTrain)

RandomForestClassifier(random_state=42)

In [ ]:
isFraudTestTree = tree.predict(paymentTest)


In [ ]:
print(isFraudTestTree)

[0 0 0 ... 0 0 0]
